In [1]:
import pennylane as qml
import pennylane.numpy as np
import jax.numpy as jnp

import jax
import optax

from time import time
import pickle

import matplotlib.pyplot as plt

import os, sys

parent = os.path.abspath('../src')
sys.path.insert(1, parent)

from perceptrons import NativePerceptron
from perceptrons import Perceptron

# Set to float64 precision and remove jax CPU/GPU warning
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")


In [2]:
# setting up the problem t=100
perceptron_qubits = 4
pulse_basis = 5*perceptron_qubits
ts = jnp.array([1.0])
t = 1

dev = qml.device("default.qubit.jax", wires = perceptron_qubits)


perceptron = Perceptron(perceptron_qubits, pulse_basis, basis='fourier', pulse_width=0.005)

H =  perceptron.H

H_obj, H_obj_spectrum = perceptron.get_1d_ising_hamiltonian(0.1)

# e_ground_state_exact = H_obj_spectrum[0]

print(f'Ising Model Hamiltonian:\nH = {H_obj}')
# print(f'Exact ground state energy: {e_ground_state_exact}')


Ising Model Hamiltonian:
H =   (0.1) [X0]
+ (0.1) [X1]
+ (0.1) [X2]
+ (0.1) [X3]
+ (1.0) [Z0 Z1]
+ (1.0) [Z1 Z2]
+ (1.0) [Z2 Z3]


I0000 00:00:1718936623.049571       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
/Users/jorgegarciaponce/Research/Yelin/PerceptronLossLandscapes/.venv/lib/python3.11/site-packages/pennylane/ops/functions/eigvals.py:108: UserWarning: For Hamiltonians, the eigenvalues will be computed numerically. This may be computationally intensive for a large number of wires. Consider using a sparse representation of the Hamiltonian with qml.SparseHamiltonian.
  warnings.warn(


In [3]:
V = qml.matrix(qml.evolve(H_obj, 1))

@jax.jit
def loss(param_vector):

    param_list = perceptron.vector_to_hamiltonian_parameters(param_vector)

    U = qml.matrix(qml.evolve(perceptron.H)(param_list, t))

    return qml.math.frobenius_inner_product(jnp.conjugate(U-V),U-V).real

In [4]:
# getting the loss_function
# loss = get_loss_function(perceptron, ts, H_obj, dev)

param_vector = perceptron.get_random_parameter_vector(89889)

print(f'Initial parameters: {param_vector}')

print(f'Initial loss: {loss(param_vector)}')

initial_gradients = jax.grad(loss)(param_vector)
print(f'Initial gradients: {initial_gradients}')

value_and_grad = jax.jit(jax.value_and_grad(loss))

Initial parameters: [7.87295744e-01 5.94898455e-01 6.53861840e-01 3.45156268e-01
 5.82579935e-01 7.43817494e-01 5.73477046e-01 2.51486620e-01
 6.82948569e-01 9.26985092e-01 1.60997814e-01 1.29324229e-01
 8.73988602e-01 3.17045967e-01 6.50151610e-01 9.72550096e-01
 6.37145484e-01 3.53570261e-02 8.21638642e-03 8.88772417e-01
 2.65338786e-01 5.15123692e-02 5.58820762e-01 1.04818365e-01
 1.95506851e-01 1.91582192e-01 4.53149813e-01 6.76653178e-01
 3.12445134e-01 7.56367447e-01 4.70969423e-02 7.75635973e-01
 3.92395182e-02 4.50542383e-01 9.24587148e-01 1.86644855e-01
 5.02370448e-01 6.79238543e-01 4.55239560e-01 7.17337328e-01
 8.42706149e-01 8.37529591e-01 5.45093815e-01 3.65413927e-01
 2.05903123e-01 2.92698707e-01 1.09554880e-01 8.62253155e-01
 4.88574939e-01 2.41067471e-01 5.70158742e-01 4.49257641e-01
 9.40087155e-01 1.43365246e-01 1.34537239e-01 6.45260926e-01
 2.76749519e-04 7.90694591e-01 7.37056177e-01 1.20429497e-01
 5.27217717e-01 8.03515002e-02 6.50296139e-01 6.89469215e-01
 8.3

In [7]:
# from datetime import datetime

n_epochs = 1000
param_vector = perceptron.get_random_parameter_vector(656)


# The following block creates a constant schedule of the learning rate
# # that increases from 0.1 to 0.5 after 10 epochs
# schedule0 = optax.constant_schedule(0.1)
# schedule1 = optax.constant_schedule(0.05)
# # schedule2 = optax.constant_schedule(0.001)
# # schedule = optax.join_schedules([schedule0, schedule1, schedule2], [200, 3000])
# schedule = optax.join_schedules([schedule0, schedule1], [200])

# optimizer = optax.adam(learning_rate=schedule)

# optimizer = optax.adam(learning_rate=0.1)

optimizer = optax.adam(learning_rate=0.1)

# optimizer = optax.sgd(learning_rate=0.005)
# optimizer = optax.adabelief(0.1)
opt_state = optimizer.init(param_vector)

energies = np.zeros(n_epochs )
# energy[0] = loss(param_vector)
mean_gradients = np.zeros(n_epochs)

gradients_trajectory = []
param_trajectory = []

# ## Compile the evaluation and gradient function and report compilation time
# time0 = time()
# _ = value_and_grad(param_vector)
# time1 = time()

# print(f"grad and val compilation time: {time1 - time0}")


## Optimization loop
for n in range(n_epochs):
    val, grads = value_and_grad(param_vector)
    updates, opt_state = optimizer.update(grads, opt_state)

    mean_gradients[n] = np.mean(np.abs(grads))
    energies[n] = val
    param_trajectory.append(param_vector)
    gradients_trajectory.append(grads)

    param_vector = optax.apply_updates(param_vector, updates)

    print(val)

    # if not n % 10:
    #     print(f"{n+1} / {n_epochs}; Frobenius norm: {val}")
    #     print(f"    mean grad: {mean_gradients[n]}")
    #     print(f'    gradient norm: {jnp.linalg.norm(grads)}')
    #     if n>=2:
    #         print(f'    difference of gradients: {jnp.linalg.norm(grads-gradients_trajectory[-2])}')



print(f"Optimal Frobenius Norm Found: {energies[-1]}")


28.14068625071369
18.12190898602038
22.785044375918194
21.704094594740706
17.610825123850717
17.246274920620188
18.396917411509694
18.644828971456473
18.03015174868321
17.107108642573362
16.74420139762736
16.49091184298514
16.273881084894573
16.235847442780134
16.327508267555444
16.330041792543117
16.031956989634985
15.451982443953103
15.15835736515572
15.4935725947039
15.698529842910363
15.577294430862779
15.268738359881398
14.911865591553225
14.951104884208304
15.181844021157387
15.144749595960509
15.025835056819696
14.899106403707744
14.797219445646366
14.866051501234889
14.876104759284676
14.811767193574758
14.820303286586231
14.760572905390163
14.72198961942176
14.727921725195886
14.701191735087956
14.731831827136286
14.716059555573036
14.652818990766937
14.660767745836797
14.664944487472146
14.678094004145802
14.666850804862563
14.624898014016525
14.639326365161097
14.647653519419087
14.64601651005345
14.633716131333621
14.615148018270691
14.631241581309203
14.62960045761091
14.6

In [6]:
param_vector.shape

(260,)